# Data Pre-Processing – Spark

Second User Testing  |  08.05.2020  |  Marc Biemer

---


The following script is used to preprocess data collected from 9 trials (18 testers) using SPARK during March 2020. 

## 1. Setup

### Import libraries

In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import statistics
import datetime

from sklearn.preprocessing import MinMaxScaler

### Define Trials & Events

The following trials needed to be excluded:
* `Trial_5` (Broken Devices, needed to be restarted, could not finish the session)
* `Trial_9` (Teacher filled the gap for a missing player)

The following events needed to be excluded due to wrong logging:
* `Planet.MultiPlayer.Move` (Logger was misplaced and therfore not fired)
* `Asteroid.MultiPlayer.Connection` (does not provide any additional value at the moment)

In [2]:
# Set relevant trials
trials = [1,2,3,4,6,7,8,10,11]

# Set relevant player colors
players = ["Red Player", "Blue Player"]

# Set relevant events
events = [
    "WhiteHole.SinglePlayer.Touch", 
    "WhiteHole.SinglePlayer.Exit",
    "WhiteHole.SinglePlayer.Trigger",
    "WhiteHole.MultiPlayer.Trigger",
    "Planet.Creation",
    "Planet.SinglePlayer.Connection",
    "Planet.SinglePlayer.Exit",
#     "Planet.MultiPlayer.Move",
    "Planet.Multiplayer.AddedToOrbit",
    "Star.Creation",
    "BlackHole.Creation",
    "BlackHole.SinglePlayer.Exit",
    "BlackHole.Multiplayer.Enter",
    "BlackHole.Transition",
    "BlackHole.TransferObject",
    "Asteroid.SinglePlayer.Connection",
    "Asteroid.SinglePlayer.Exit",
#     "Asteroid.MultiPlayer.Connection"
]

## 2. User Engagement Scores & Behavioral Intention Scores

In [17]:
# Import questionnaire data
questionnaire = pd.read_csv("data/questionnaire.csv")
questionnaire = questionnaire.iloc[:, 1:]

# Filter Trials
questionnaire = questionnaire.loc[questionnaire['Trial ID'].isin(trials)]
 

## UES SCORES
# Calculate questionnaire subscores
for col in ["FA", "PU", "AE", "RW"]:
    questionnaire[col] = questionnaire.loc[:,[col+"-S1",col+"-S2",col+"-S3"]].mean(axis=1)

# Invert rating for PU
questionnaire.loc[:, "PU"] = 6 - questionnaire.loc[:, "PU"]

# Calculate total UES
questionnaire["UES"] = questionnaire.loc[:,["FA","PU","AE","RW"]].mean(axis=1)

# Export to csv
ues = questionnaire.copy()
ues.pop("BIS-S1")
ues.to_csv(r'preprocessed_data/UES.csv', index = False)
# print(ues.head())


## BIS SCORE
# Clean up dataframe
bis = questionnaire.copy()
for col in ["FA-S1","FA-S2","FA-S3","PU-S1","PU-S2","PU-S3","AE-S1","AE-S2","AE-S3","RW-S1","RW-S2","RW-S3","FA","PU","AE","RW","UES"]:
    bis.pop(col)
bis = bis.rename(columns={"BIS-S1": "BIS"}, errors="raise")

# Export to csv
bis.to_csv(r'preprocessed_data/BIS.csv', index = False)
# print(bis.head())

## 3. User Movement

In [ ]:
# !!!! DO NOT RE-EXECUTE !!!!
# NEEDS TO BE EXECUTE ONLY ON FIRST USE, IF PREPROCESSED DATA IS NOT AVAILABLE

# Define import function
def importPositions(trialId):
    trial_data = pd.read_json("data/logFiles/position/trial_" + str(trialId) + ".log")
    
    # Format position data to int values, dropping decimals
    for row in range(trial_data.shape[0]):
        trial_data.loc[row, "Position.X"] = int((trial_data.loc[row,"Position.X"].split(',')[0]))
        trial_data.loc[row, "Position.Y"] = int((trial_data.loc[row,"Position.Y"].split(',')[0]))
    
    # Create a complex number for each position
    trial_data["Pos"] = trial_data["Position.X"] + 1j*trial_data["Position.Y"]
    
    return trial_data

# Export each trial to CSV
for trial in trials:
    export = pd.DataFrame(importPositions(trial))
    export.to_csv(r'preprocessed_data/movement/trial_'+ str(trial) + '.csv', index = False)

## 4. Events

In [53]:
# Fetch start and times for each trial
start_end_times = pd.DataFrame()
start_end_times["Trial"] = ""
start_end_times["Time.Start"] = ""
start_end_times["Time.End"] = ""

# Import "compressed" movement data
def importPosData(trialId):
    data = pd.read_csv("preprocessed_data/movement/trial_" + str(trialId) + ".csv")
    return data

# Find start and end times for each trial
for trial in trials:
    log = importPosData(trial)
    start = datetime.datetime.strptime(log.loc[0, "Time"], '%Y/%m/%d %H:%M:%S.%f')
    last = log.shape[0] - 1
    end = datetime.datetime.strptime(log.loc[last, "Time"], '%Y/%m/%d %H:%M:%S.%f')
    start_end_times = start_end_times.append({'Trial': trial, 'Time.Start': start, 'Time.End': end}, ignore_index=True)

In [54]:
# Support function to find the right Position.X column for each event
def getColX (arg):
    switcher = {
        
        "WhiteHole.SinglePlayer.Touch": "WhiteHole.Position.X", 
        "WhiteHole.SinglePlayer.Exit": "WhiteHole.Position.X",
        "WhiteHole.SinglePlayer.Trigger": "WhiteHole.Position.X",
        
        "Planet.Creation": "Planet.Position.X",
        "Planet.SinglePlayer.Connection": "Planet.Position.X",
        "Planet.SinglePlayer.Exit": "Planet.Position.X",
        "Planet.Multiplayer.AddedToOrbit": "Planet.Position.X",
        
        "Star.Creation": "Star.Position.X",
        
        "BlackHole.Creation": "BlackHole.Position.X",
        "BlackHole.SinglePlayer.Exit": "BlackHole.Position.X",
        "BlackHole.Multiplayer.Enter": "BlackHole.Position.X",
        "BlackHole.Transition": "BlackHole.Position.X",
        "BlackHole.TransferObject": "BlackHole.Position.X",
        
        "Asteroid.SinglePlayer.Connection": "Asteroid.Position.X",
        "Asteroid.SinglePlayer.Exit": "Asteroid.Position.X",
        "Asteroid.MultiPlayer.Connection": "Asteroid.Position.X",

    }
    return switcher.get(arg, np.nan)

# Support function to find the right Position.Y column for each event
def getColY (arg):
    switcher = {
        
        "WhiteHole.SinglePlayer.Touch": "WhiteHole.Position.Y", 
        "WhiteHole.SinglePlayer.Exit": "WhiteHole.Position.Y",
        "WhiteHole.SinglePlayer.Trigger": "WhiteHole.Position.Y",
        
        "Planet.Creation": "Planet.Position.Y",
        "Planet.SinglePlayer.Connection": "Planet.Position.Y",
        "Planet.SinglePlayer.Exit": "Planet.Position.Y",
        "Planet.Multiplayer.AddedToOrbit": "Planet.Position.Y",
        
        "Star.Creation": "Star.Position.Y",
        
        "BlackHole.Creation": "BlackHole.Position.Y",
        "BlackHole.SinglePlayer.Exit": "BlackHole.Position.Y",
        "BlackHole.Multiplayer.Enter": "BlackHole.Position.Y",
        "BlackHole.Transition": "BlackHole.Position.Y",
        "BlackHole.TransferObject": "BlackHole.Position.Y",
        
        "Asteroid.SinglePlayer.Connection": "Asteroid.Position.Y",
        "Asteroid.SinglePlayer.Exit": "Asteroid.Position.Y",
        "Asteroid.MultiPlayer.Connection": "Asteroid.Position.Y",

    }
    return switcher.get(arg, np.nan)

In [55]:
# Restructure Event Data and Initial Cleanup of too often logged Events

# Import each trial individually
for trial in trials:
    log = pd.read_json("data/logFiles/events/trial_" + str(trial) + ".log")
    
    # Fetch start and end times
    start = start_end_times.loc[start_end_times["Trial"] == trial,"Time.Start"].values[0]
    end = start_end_times.loc[start_end_times["Trial"] == trial,"Time.End"].values[0]
    
    # Prepare export DataFrame to hold clean data
    export = pd.DataFrame()
    export["Time"] = ""
    export["Trial"] = ""
    export["Event"] = ""
    export["Player.Color"] = ""
    export["Position.X"] = ""
    export["Position.Y"] = ""
    export["Planet.Id"] = ""

    # Add Start Event to new export
    export = export.append({'Trial': trial, 'Time': start, 'Event': "Start"}, ignore_index=True)
    
    # Loop through rows to find relevant events
    for row in range(log.shape[0]):
        
        rel = True;
        t = datetime.datetime.strptime(log.loc[row, "Time"], '%Y/%m/%d %H:%M:%S.%f')
        e = log.loc[row,"Event"]
        p = log.loc[row,"Player.Color"]
        
        # Ignore events which are not defined as meaningful
        if (e not in events):
            rel = False
        
        if (e.startswith("Planet")):
            p_id = log.loc[row, "Planet.Id"].split("-")[1]
            
            if (e == "Planet.SinglePlayer.Exit"):
                done = False
                i = 1
                while not done:
                    if ( row > 0 and row-i >= 0):
                        last_e = log.loc[row-i,"Event"]
                        if (last_e.startswith('Planet.SinglePlayer.') and log.loc[row-i, "Player.Color"] == p):
                            if (last_e == 'Planet.SinglePlayer.Connection'):
#                             if (last_e == 'Planet.SinglePlayer.Connection' and log.loc[row-i, "Planet.Id"].split("-")[1] == p_id):
                                rel = True
                            else:
                                rel = False
                            done = True;
                        else:
                            i += 1
                    else:
                        done = True
                        rel = False            
        else:
            p_id = np.nan
            
        if (e == "Asteroid.SinglePlayer.Connection"):
            done = False
            i = 1
            while not done:
                if ( row > 0 and row-i >= 0):
                    last_e = log.loc[row-i,"Event"]
                    if (last_e.startswith('Asteroid') and log.loc[row-i, "Player.Color"] == p):
                        if (last_e == 'Asteroid.SinglePlayer.Connection'):
                            rel = False
                        else:
                            rel = True
                        done = True;
                    else:
                        i += 1
                else:
                    done = True
                    rel = True        
                       
        if (e == "WhiteHole.MultiPlayer.Trigger"):
            posX = round((int(log.loc[row, "WhiteHole.1.Position.X"].split(",")[0]) + int(log.loc[row, "WhiteHole.2.Position.X"].split(",")[0]))/2)
            posY = round((int(log.loc[row, "WhiteHole.1.Position.Y"].split(",")[0]) + int(log.loc[row, "WhiteHole.2.Position.Y"].split(",")[0]))/2)
        else:
            posX = int(log.loc[row, getColX(e)].split(",")[0])
            posY = int(log.loc[row, getColY(e)].split(",")[0])

        # Add event line to export if relevant
        if (rel):
            export = export.append({'Trial': trial, 
                                    'Time': t, 
                                    'Event': e, 
                                    'Player.Color': p, 
                                    'Planet.Id': p_id, 
                                    "Position.X": posX, 
                                    "Position.Y": posY }, ignore_index=True)
    # Add End event          
    export = export.append({'Trial': trial, 'Time': end, 'Event': "End"}, ignore_index=True)
    
    # Export trial to new CSV
    export.to_csv(r'preprocessed_data/events/trial_'+ str(trial) + '.csv', index = False)

In [56]:
# Re-import preproccessed files
def importEventData(trialId):
    data = pd.read_csv("preprocessed_data/events/trial_" + str(trialId) + ".csv")
    for row in range(data.shape[0]):
        data.loc[row, "Time"] = datetime.datetime.strptime(data.loc[row,"Time"], '%Y-%m-%d %H:%M:%S.%f')
    return data

trials_events = []
    
for trial in trials:
    trials_events.append(importEventData(trial))

In [57]:
# Fix Meaningful Count of Asteroid Interactions

# Loop through each trial
i = 1
for trial in trials_events:
    
    # Adjust i for later labeling of trials (5 and 9 were excluded before)
    if (i==5 or i==9):
        i += 1
    
    # Go through all rows for each player
    for player in players:
        
        done = False
        row = 1
        
        while not done:
            
            # Find events for connecting to Asteroids
            if (trial.loc[row, "Player.Color"] == player and trial.loc[row,"Event"] == "Asteroid.SinglePlayer.Connection"):
                e = trial.loc[row,"Event"]
                

                # Look for previous events
                j = 1
                last = False
                while not last:
                    if (row > 0 and row-j >= 0):

                        # Check for previous event of player
                        if (trial.loc[row-j, "Player.Color"] == player):
                            last_e = trial.loc[row-j, "Event"]

                            # Case 1: Last event was not related to an Asteroid
                            if (not str(last_e).startswith('Asteroid')):
                                last = True

                            # Case 2: Last event was Asteroid_Exit
                            elif (last_e == "Asteroid.SinglePlayer.Exit"):                                    

                                # Check if last Connection is older than 2 sec
                                if ((trial.loc[row-j, "Time"] + datetime.timedelta(seconds=2.0)) > trial.loc[row, "Time"]):

                                    # Delete new connection event
                                    trial.loc[row, "Event"] = np.nan

                                    # Delete previous exit event
                                    trial.loc[row-j, "Event"] = np.nan

                                last = True

                            else:
                                j += 1
                        else:
                            j += 1
                    else:
                        last = True
            elif(trial.loc[row,"Event"] == "End"):
                done = True
            
            row += 1
                        
    
    # Remove NaN events
    trial = trial.loc[trial["Event"].notnull()]
    
    # Export
    trial.to_csv(r'preprocessed_data/events/trial_'+ str(i) + '.csv', index = False)
    print("Done with Trial_" + str(i))
    i += 1

Done with Trial_1
Done with Trial_2
Done with Trial_3
Done with Trial_4
Done with Trial_6
Done with Trial_7
Done with Trial_8
Done with Trial_10
Done with Trial_11


In [ ]:
# Re-import preproccessed files
def importEventData(trialId):
    data = pd.read_csv("preprocessed_data/events/trial_" + str(trialId) + ".csv")
    for row in range(data.shape[0]):
        data.loc[row, "Time"] = datetime.datetime.strptime(data.loc[row,"Time"], '%Y-%m-%d %H:%M:%S.%f')
    return data

In [ ]:
# # Code to fix missing Planet.SinglePlayer.Exit statements
# ## Trial 7
# trial = importEventData(7)
# #Missing events
# T7 
# Blue
# 11:14:30
# Red
# 11:07:00
# trial = insert_row(idx, trial, missingEvent)
# print(trial)

# ## Events to add for trial 8
# #

# ## Events to add for trial 10
# #

In [58]:
# Restructure Events to present boolean states for each interaction type and player
trials_events = []
    
for trial in trials:
    trials_events.append(importEventData(trial))

In [59]:
# Loop through each trial
i = 1
for trial in trials_events:
    
    # Adjust i for later labeling of trials (5 and 9 were excluded before)
    if (i==5 or i==9):
        i += 1
    
    # Initiate all states with 0
    trial["WhiteHole.Interaction.BluePlayer"] = 0
    trial["WhiteHole.Interaction.RedPlayer"] = 0
    trial["Planet.Interaction.BluePlayer"] = 0
    trial["Planet.Interaction.RedPlayer"] = 0
    trial["Asteroid.Interaction.BluePlayer"] = 0
    trial["Asteroid.Interaction.RedPlayer"] = 0
   
    # Support for looping through players
    wh_int = ["WhiteHole.Interaction.RedPlayer", "WhiteHole.Interaction.BluePlayer"]
    pl_int = ["Planet.Interaction.RedPlayer", "Planet.Interaction.BluePlayer"]
    ast_int = ["Asteroid.Interaction.RedPlayer", "Asteroid.Interaction.BluePlayer"]
    
    # Check if player is interacting at each time stamp
    p = 0
    for player in players:
        
        wh_logger = False
        pl_logger = False
        ast_logger = False
        
        for row in range(trial.shape[0]):
            
            # With WhiteHoles
            if (trial.loc[row, "Event"] == "WhiteHole.SinglePlayer.Touch" and trial.loc[row, "Player.Color"] == player):
                wh_logger = True
            if (wh_logger):
                trial.loc[row, wh_int[p]] = 1
            if ((trial.loc[row, "Event"] == "WhiteHole.SinglePlayer.Trigger" or trial.loc[row, "Event"] == "WhiteHole.SinglePlayer.Exit") and trial.loc[row, "Player.Color"] == player):
                wh_logger = False
            
            # With Planets
            if (trial.loc[row, "Event"] == "Planet.SinglePlayer.Connection" and trial.loc[row, "Player.Color"] == player):
                pl_logger = not pl_logger
            if (pl_logger):
                trial.loc[row, pl_int[p]] = 1
            if ((trial.loc[row, "Event"] == "Planet.SinglePlayer.Exit") and trial.loc[row, "Player.Color"] == player):
                pl_logger = not pl_logger
        
            # With Asteroids
            if (trial.loc[row, "Event"] == "Asteroid.SinglePlayer.Connection" and trial.loc[row, "Player.Color"] == player):
                ast_logger = not ast_logger
            if (ast_logger):
                trial.loc[row, ast_int[p]] = 1
            if ((trial.loc[row, "Event"] == "Asteroid.SinglePlayer.Exit") and trial.loc[row, "Player.Color"] == player):
                ast_logger = not ast_logger
        
        p += 1

    trial.to_csv(r'preprocessed_data/events/bool/trial_'+ str(i) + '.csv', index = False)
    i += 1

In [60]:
def createEmpty(df, row):
    empty = pd.DataFrame(df.loc[row:row+1,:])
    empty = empty.drop(empty.index[1])
    empty["Time"] = empty["Time"] + datetime.timedelta(seconds=.01)
    empty["Event"] = np.nan
    empty["Player.Color"] = np.nan
    empty["Position.X"] = np.nan
    empty["Position.Y"] = np.nan
    empty["Planet.Id"] = np.nan
    return empty

def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]

    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)

    return df

In [61]:
# Restructure Events to present boolean states for each interaction type and player

# Re-import preproccessed files
def importEventData(trialId):
    data = pd.read_csv("preprocessed_data/events/bool/trial_" + str(trialId) + ".csv")
    for row in range(data.shape[0]):
        data.loc[row, "Time"] = datetime.datetime.strptime(data.loc[row,"Time"], '%Y-%m-%d %H:%M:%S.%f')
    return data

trials_events = []
    
for trial in trials:
    trials_events.append(importEventData(trial))

In [62]:
# Loop through each trial
i = 1
for trial in trials_events:
    
    # Adjust i for later labeling of trials (5 and 9 were excluded before)
    if (i==5 or i==9):
        i += 1
    
    done = False
    row = 0
    while not done:
        
        if (trial.loc[row, "Event"] == "End"):
            done = True
        
        if (str(trial.loc[row, "Event"]).endswith(".Exit") or str(trial.loc[row, "Event"]).endswith(".Trigger")):
            
            empty = createEmpty(trial, row)
            
            if (trial.loc[row, "Event"] == "WhiteHole.SinglePlayer.Exit" or trial.loc[row, "Event"] == "WhiteHole.SinglePlayer.Trigger"):
                if (trial.loc[row, "Player.Color"] == "Red Player"):
                    empty["WhiteHole.Interaction.RedPlayer"] = 0
                else: 
                    empty["WhiteHole.Interaction.BluePlayer"] = 0
            elif (trial.loc[row, "Event"] == "Planet.SinglePlayer.Exit"):
                if (trial.loc[row, "Player.Color"] == "Red Player"):
                    empty["Planet.Interaction.RedPlayer"] = 0
                else: 
                    empty["Planet.Interaction.BluePlayer"] = 0
            elif (trial.loc[row, "Event"] == "Asteroid.SinglePlayer.Exit"):
                if (trial.loc[row, "Player.Color"] == "Red Player"):
                    empty["Asteroid.Interaction.RedPlayer"] = 0
                else: 
                    empty["Asteroid.Interaction.BluePlayer"] = 0
            
            trial = insert_row(row+1, trial, empty)
        
        row += 1

    trial.to_csv(r'preprocessed_data/events/bool/trial_'+ str(i) + '.csv', index = False)
    i += 1